In [ ]:
from argparse import Namespace
import time
import random

from my_code.data_generation import generate_tables, generate_tables2
from my_code.blocking import run_blocking
from my_code.summarizer import Summarizer
from my_code.DK import ProductDKInjector, GeneralDKInjector
from my_code.dataset import DittoDataset
from my_code.utils import train
from my_code.utils import plot_metrics, normalize_columns, fetch_from_hive

In [ ]:
def run_full_pipeline(hp, config):
    trainset = config['trainset']
    validset = config['validset']
    testset = config['testset']
    random.seed(42)  # For reproducibility
    
    run_blocking(hp)
    if hp.summarize:
        summarizer = Summarizer(config, hp.lm)
        trainset = summarizer.transform_file(trainset, max_len = hp.max_len, overwrite=hp.overwrite)
        testset = summarizer.transform_file(testset, max_len = hp.max_len, overwrite=hp.overwrite)
        validset = summarizer.transform_file(validset, max_len = hp.max_len, overwrite=hp.overwrite)

    if hp.dk is not None:
        if hp.dk == 'product':
            injector = ProductDKInjector(config, hp.dk)
        else:
            injector = GeneralDKInjector(config, hp.dk)
        
        trainset = injector.transform_file(trainset, overwrite=hp.overwrite)
        validset = injector.transform_file(validset, overwrite=hp.overwrite)
        testset = injector.transform_file(testset, overwrite=hp.overwrite)

    train_dataset = DittoDataset(trainset,
                                   lm=hp.lm,
                                   max_len=hp.max_len,
                                   size=hp.size,
                                   da=hp.da)
    valid_dataset = DittoDataset(validset, lm=hp.lm)
    test_dataset = DittoDataset(testset, lm=hp.lm)

    t1 = time.time()
    csv_log_path = train(train_dataset, valid_dataset, test_dataset, run_tag="test_run", hp=hp)
    t2 = time.time()

    print(f"Training time: {round(t2-t1, 3)} seconds")
    plot_metrics(csv_log_path)

In [ ]:
def main(hp, config):
    #---------------------------------------------------------------------------------------------
    #TO UPDATE IF NEEDED FOR DATA GENERATION
    if hp.task == "Generated_data":
        #generate_tables(base_path=hp.base_path_blocking, n_total=100, match_ratio=0.3)
        generate_tables2(base_path=hp.base_path_blocking, n_total=100, match_ratio=0.3)
        normalize_columns(df_path=hp.table_reference_csv, json_file="my_code/mapping.json")
        normalize_columns(df_path=hp.table_source_csv, json_file="my_code/mapping.json")
    #---------------------------------------------------------------------------------------------
    #If the data already exist make sure to add the task name to configs and also use it hp.task and put the data in base_path_blocking/data/{task_name}   (reference and source)
    #---------------------------------------------------------------------------------------------
    # Case 2: Use Hive to fetch existing data
    else:
        fetch_from_hive(
            hive_host=hp.hive_host,
            hive_port=hp.hive_port,
            hive_user=hp.hive_user,
            hive_database=hp.hive_database,
            source_table=hp.source_table,
            reference_table=hp.reference_table,
            source_csv_path=hp.table_source_csv,
            reference_csv_path=hp.table_reference_csv
        )

        # Normalize after fetching
        normalize_columns(df_path=hp.table_reference_csv, json_file="my_code/mapping.json")
        normalize_columns(df_path=hp.table_source_csv, json_file="my_code/mapping.json")
    run_full_pipeline(hp, config)

In [ ]:
# ---------------------------------------------------------------------------------------------
# Base path for storing generated or fetched datasets
base_path_blocking = "D:/Study/ENSIAS/stage_2/ER/ditto/resultat"  # Update to your project root

# -------------------- Task Selection --------------------
# Choose the task/dataset you want to work with:
# - "Generated_data" → Use synthetic data generated by the script
# - Any other name → Use your own dataset fetched from Hive
task = "Generated_data"

# -------------------- Hive Configuration --------------------
# If you want to fetch your dataset from Hive, provide these parameters.
# Otherwise, keep them as None when using generated data.
hive_host = None        # Hive server host
hive_port = None        # Hive server port
hive_user = None        # Hive username
hive_database = None    # Hive database name
source_table = None     # Hive source table name
reference_table = None  # Hive reference table name
# ---------------------------------------------------------------------------------------------

hp = Namespace(
    # -----------------------------------------------------------------------------------------
    #Hive configuration
    hive_host = hive_host,             # Hive server host
    hive_port = hive_port,             # Hive server port
    hive_user = hive_user,             # Hive username
    hive_database = hive_database,     # Hive database name
    source_table = source_table,       # Hive source table name
    reference_table = reference_table, # Hive reference table name
    # -----------------------------------------------------------------------------------------

    # -----------------------------------------------------------------------------------------
    # Hyperparameters for blocking part
    model_name_blocking="all-MiniLM-L12-v2",
    top_k_blocking=5,
    threshold_blocking=0.95,
    batch_size_blocking=512,
    # -----------------------------------------------------------------------------------------

    # Paths
    base_path_blocking=base_path_blocking,

    # Input CSVs
    table_reference_csv=f"{base_path_blocking}/data/{task}/reference_table.csv",
    table_source_csv=f"{base_path_blocking}/data/{task}/source_table.csv",
    ground_truth_csv=f"{base_path_blocking}/data/{task}/ground_truth.csv",

    # Ditto-style TXT
    table_reference_txt=f"{base_path_blocking}/input_txt_blocking/{task}/reference_table.txt",
    table_source_txt=f"{base_path_blocking}/input_txt_blocking/{task}/source_table.txt",

    # Vector files
    table_reference_vec=f"{base_path_blocking}/vectors_blocking/{task}/reference_table.txt.mat",
    table_source_vec=f"{base_path_blocking}/vectors_blocking/{task}/source_table.txt.mat",

    # Blocking outputs
    output_pairs_csv=f"{base_path_blocking}/blocking/{task}/blocking_pairs.csv",
    output_ditto_txt=f"{base_path_blocking}/blocking/{task}/blocking_pairs_ditto.txt",

    # Inference output
    output_inference_csv=f"{base_path_blocking}/inference/{task}/result.csv",

    dataset_csv_dir=f"{base_path_blocking}/dataset_ditto_csv",
    dataset_txt_dir=f"{base_path_blocking}/dataset_ditto_txt",

    # Logging and task info
    logdir="./logs",
    task=task,

    # -----------------------------------------------------------------------------------------
    # Hyperparameters for training
    batch_size=32,
    lr=3e-5,
    epochs=5,
    save_model=True,
    lm="distilbert",
    size=None,
    alpha_aug=0.8,
    max_len=256,
    da="all",
    summarize=True,
    dk=True,
    fp16=True,
    overwrite=True,
    columns_to_use = None #Column to use (None means all columns)
    # -----------------------------------------------------------------------------------------
)

# The `configs` list tells the project where to find train/validation/test data
# Each task needs a dictionary with the paths to its dataset files.
# Example structure:
configs = [{
    "name": "Generated_data",
    "trainset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/train.txt",
    "validset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/valid.txt",
    "testset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/test.txt"
},
{
    "name": "data_1",
    "trainset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/train.txt",
    "validset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/valid.txt",
    "testset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/test.txt"
}
]

configs = {conf['name'] : conf for conf in configs}
config = configs[hp.task]

main(hp, config)

In [ ]:
# ---------------------------------------------------------------------------------------------
# Base path for storing generated or fetched datasets
base_path_blocking = "D:/Study/ENSIAS/stage_2/ER/ditto/resultat"  # Update to your project root

# -------------------- Task Selection --------------------
# Choose the task/dataset you want to work with:
# - "Generated_data" → Use synthetic data generated by the script
# - Any other name → Use your own dataset fetched from Hive
task = "Generated_data"

# -------------------- Hive Configuration --------------------
# If you want to fetch your dataset from Hive, provide these parameters.
# Otherwise, keep them as None when using generated data.
hive_host = None        # Hive server host
hive_port = None        # Hive server port
hive_user = None        # Hive username
hive_database = None    # Hive database name
source_table = None     # Hive source table name
reference_table = None  # Hive reference table name
# ---------------------------------------------------------------------------------------------

hp = Namespace(
    # -----------------------------------------------------------------------------------------
    #Hive configuration
    hive_host = hive_host,             # Hive server host
    hive_port = hive_port,             # Hive server port
    hive_user = hive_user,             # Hive username
    hive_database = hive_database,     # Hive database name
    source_table = source_table,       # Hive source table name
    reference_table = reference_table, # Hive reference table name
    # -----------------------------------------------------------------------------------------

    # -----------------------------------------------------------------------------------------
    # Hyperparameters for blocking part
    model_name_blocking="all-MiniLM-L12-v2",
    top_k_blocking=5,
    threshold_blocking=0.95,
    batch_size_blocking=512,
    # -----------------------------------------------------------------------------------------

    # Paths
    base_path_blocking=base_path_blocking,

    # Input CSVs
    table_reference_csv=f"{base_path_blocking}/data/{task}/reference_table.csv",
    table_source_csv=f"{base_path_blocking}/data/{task}/source_table.csv",
    ground_truth_csv=f"{base_path_blocking}/data/{task}/ground_truth.csv",

    # Ditto-style TXT
    table_reference_txt=f"{base_path_blocking}/input_txt_blocking/{task}/reference_table.txt",
    table_source_txt=f"{base_path_blocking}/input_txt_blocking/{task}/source_table.txt",

    # Vector files
    table_reference_vec=f"{base_path_blocking}/vectors_blocking/{task}/reference_table.txt.mat",
    table_source_vec=f"{base_path_blocking}/vectors_blocking/{task}/source_table.txt.mat",

    # Blocking outputs
    output_pairs_csv=f"{base_path_blocking}/blocking/{task}/blocking_pairs.csv",
    output_ditto_txt=f"{base_path_blocking}/blocking/{task}/blocking_pairs_ditto.txt",

    # Inference output
    output_inference_csv=f"{base_path_blocking}/inference/{task}/result.csv",

    dataset_csv_dir=f"{base_path_blocking}/dataset_ditto_csv",
    dataset_txt_dir=f"{base_path_blocking}/dataset_ditto_txt",

    # Logging and task info
    logdir="./logs",
    task=task,

    # -----------------------------------------------------------------------------------------
    # Hyperparameters for training
    batch_size=32,
    lr=3e-5,
    epochs=5,
    save_model=True,
    lm="distilbert",
    size=None,
    alpha_aug=0.8,
    max_len=256,
    da="all",
    summarize=True,
    dk=True,
    fp16=True,
    overwrite=True,
    columns_to_use = ["ifu", "nom", "prenoms", "raison_sociale"] #Column to use (None means all columns)
    # -----------------------------------------------------------------------------------------
)

# The `configs` list tells the project where to find train/validation/test data
# Each task needs a dictionary with the paths to its dataset files.
# Example structure:
configs = [{
    "name": "Generated_data",
    "trainset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/train.txt",
    "validset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/valid.txt",
    "testset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/test.txt"
},
{
    "name": "data_1",
    "trainset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/train.txt",
    "validset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/valid.txt",
    "testset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/test.txt"
}
]

configs = {conf['name'] : conf for conf in configs}
config = configs[hp.task]

main(hp, config)

In [ ]:
# ---------------------------------------------------------------------------------------------
# Base path for storing generated or fetched datasets
base_path_blocking = "D:/Study/ENSIAS/stage_2/ER/ditto/resultat"  # Update to your project root

# -------------------- Task Selection --------------------
# Choose the task/dataset you want to work with:
# - "Generated_data" → Use synthetic data generated by the script
# - Any other name → Use your own dataset fetched from Hive
task = "data_1"

# -------------------- Hive Configuration --------------------
# If you want to fetch your dataset from Hive, provide these parameters.
# Otherwise, keep them as None when using generated data.
hive_host = None        # Hive server host
hive_port = None        # Hive server port
hive_user = None        # Hive username
hive_database = None    # Hive database name
source_table = None     # Hive source table name
reference_table = None  # Hive reference table name
# ---------------------------------------------------------------------------------------------

hp = Namespace(
    # -----------------------------------------------------------------------------------------
    #Hive configuration
    hive_host = hive_host,             # Hive server host
    hive_port = hive_port,             # Hive server port
    hive_user = hive_user,             # Hive username
    hive_database = hive_database,     # Hive database name
    source_table = source_table,       # Hive source table name
    reference_table = reference_table, # Hive reference table name
    # -----------------------------------------------------------------------------------------

    # -----------------------------------------------------------------------------------------
    # Hyperparameters for blocking part
    model_name_blocking="all-MiniLM-L12-v2",
    top_k_blocking=5,
    threshold_blocking=0.95,
    batch_size_blocking=512,
    # -----------------------------------------------------------------------------------------

    # Paths
    base_path_blocking=base_path_blocking,

    # Input CSVs
    table_reference_csv=f"{base_path_blocking}/data/{task}/reference_table.csv",
    table_source_csv=f"{base_path_blocking}/data/{task}/source_table.csv",
    ground_truth_csv=f"{base_path_blocking}/data/{task}/ground_truth.csv",

    # Ditto-style TXT
    table_reference_txt=f"{base_path_blocking}/input_txt_blocking/{task}/reference_table.txt",
    table_source_txt=f"{base_path_blocking}/input_txt_blocking/{task}/source_table.txt",

    # Vector files
    table_reference_vec=f"{base_path_blocking}/vectors_blocking/{task}/reference_table.txt.mat",
    table_source_vec=f"{base_path_blocking}/vectors_blocking/{task}/source_table.txt.mat",

    # Blocking outputs
    output_pairs_csv=f"{base_path_blocking}/blocking/{task}/blocking_pairs.csv",
    output_ditto_txt=f"{base_path_blocking}/blocking/{task}/blocking_pairs_ditto.txt",

    # Inference output
    output_inference_csv=f"{base_path_blocking}/inference/{task}/result.csv",

    dataset_csv_dir=f"{base_path_blocking}/dataset_ditto_csv",
    dataset_txt_dir=f"{base_path_blocking}/dataset_ditto_txt",

    # Logging and task info
    logdir="./logs",
    task=task,

    # -----------------------------------------------------------------------------------------
    # Hyperparameters for training
    batch_size=32,
    lr=3e-5,
    epochs=5,
    save_model=True,
    lm="distilbert",
    size=None,
    alpha_aug=0.8,
    max_len=256,
    da="all",
    summarize=True,
    dk=True,
    fp16=True,
    overwrite=True,
    columns_to_use = None #Column to use (None means all columns)
    # -----------------------------------------------------------------------------------------
)

# The `configs` list tells the project where to find train/validation/test data
# Each task needs a dictionary with the paths to its dataset files.
# Example structure:
configs = [{
    "name": "Generated_data",
    "trainset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/train.txt",
    "validset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/valid.txt",
    "testset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/test.txt"
},
{
    "name": "data_1",
    "trainset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/train.txt",
    "validset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/valid.txt",
    "testset": f"{hp.base_path_blocking}/dataset_ditto_txt/{hp.task}/test.txt"
}
]

configs = {conf['name'] : conf for conf in configs}
config = configs[hp.task]

main(hp, config)